# Back propagation

In [ ]:
import os

import kagglehub
import pandas as pd
from torch import nn

In [ ]:
class NeuralNetwork(nn.Module):
    """
    Feedforward Neural Network with one hidden layer.

    Architecture:
    - Input layer: n_features neurons
    - Hidden layer: n_hidden neurons (ReLU activation)
    - Output layer: n_classes neurons (logits - no softmax, CrossEntropyLoss does this)
    """

    def __init__(self, n_features, n_hidden, n_classes):
        super().__init__()

        # Define layers
        self.fc1 = nn.Linear(n_features, n_hidden)  # Input to hidden
        self.relu = nn.ReLU()  # ReLU activation
        self.fc2 = nn.Linear(n_hidden, n_classes)  # Hidden to output

        # Optional: Custom weight initialization (He initialization for ReLU)
        nn.init.kaiming_normal_(self.fc1.weight, nonlinearity="relu")
        nn.init.kaiming_normal_(self.fc2.weight, nonlinearity="relu")

    def forward(self, x):
        """Forward pass through the network."""
        x = self.fc1(x)  # Linear transformation
        x = self.relu(x)  # ReLU activation
        x = self.fc2(x)  # Linear transformation (logits)
        return x  # No softmax - CrossEntropyLoss handles it!

In [ ]:
device = "cpu"

In [ ]:
# Download data from kaggle
path = kagglehub.dataset_download("yashdevladdha/uber-ride-analytics-dashboard")

# Load data into Pandas DataFrame
csv_file = os.path.join(path, "ncr_ride_bookings.csv")
df = pd.read_csv(csv_file)
print("✅ Data loaded successfully!")

- Target variabele $\pmb{y}$:  
    - `Payment Method`
- Features $\pmb{X}$:
    - `Avg VTAT`
    - `Avg CTAT`
    - `Booking Value`
    - `Ride Distance` 

In [ ]:
df = (
    df.loc[:, ("Payment Method", "Avg VTAT", "Avg CTAT", "Booking Value", "Ride Distance")]
    .dropna()
    .reset_index(drop=True)
)

In [ ]:
print(df["Payment Method"].unique())

## ✍️
Maak NumPy array aan voor de target $\pmb{y}$.

## ✍️
Maak NumPy array aan voor de featurematrix $\pmb{X}$ mèt _standaardschaling_.

## ✍️
Maak random 80/20% train/test split van $\pmb{y}$ en $\pmb{X}$.

## ✍️
Converteer $\pmb{y}$ en $\pmb{X}$ naar PyTorch formaat.

## ✍️
Train een `NeuralNetwork` model (zie hierboven) met 10 hidden layers.

## ✍️
Definieer de _Cross Entropy Loss_ functie en _Stochastic Gradient Descent_ als optimalisatiealgoritme (_learning rate_ = 0.02)

## ✍️
Train the model for 2000 epochs and plot the test/training _loss_ en accuraatheid.